# import

In [1]:

import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *
import pickle


# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_win2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_win = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'
path_ft = '/Users/Owner/Desktop/Horse/ft_data/peds_ft.txt'

# load data

In [3]:
results = load_csv(path_win+'results.csv')
horse_results = load_csv(path_win+'horse_results.csv')
peds = load_csv(path_win+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_win+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [4]:
ll = LearnLGBM(peds,results,horse_results)
ll.path_ft = '/Users/Owner/Desktop/Horse/horse/peds_ft.txt'
ll.learn_lgb()

  0%|          | 0/18638 [00:00<?, ?it/s]

  0%|          | 0/18638 [00:00<?, ?it/s]

pe finish
pe regularizrd


  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Total Bins 47243
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172


# grid search

In [5]:
train_data = ll.r.data_c
train, valid = split_data(train_data,test_size=0.2,rank_learning=False)
# 0.8 : 0.2

x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_valid = valid.drop(['rank', 'date','単勝'], axis=1)
y_valid = valid['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
valid_query = x_valid.groupby(x_valid.index).size()
valid = lgb.Dataset(x_valid, y_valid, reference=train, group=valid_query)


def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用

    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        'num_iterations': hp.choice('num_iterations',range(50,120)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)


trials = Trials()
optimize(trials)


Training start:                                       
  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058385 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 47243                    
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172
Training until validation scores don't improve for 20 rounds
  0%|          | 0/25 [00:01<?, ?trial/s, best loss=?]

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')



[5]	valid's ndcg@1: 0.540571	valid's ndcg@2: 0.52705	valid's ndcg@3: 0.517194
[10]	valid's ndcg@1: 0.535429	valid's ndcg@2: 0.524805	valid's ndcg@3: 0.515757
[15]	valid's ndcg@1: 0.542286	valid's ndcg@2: 0.526135	valid's ndcg@3: 0.517458
[20]	valid's ndcg@1: 0.537143	valid's ndcg@2: 0.524808	valid's ndcg@3: 0.517306
Early stopping, best iteration is:                    
[1]	valid's ndcg@1: 0.556	valid's ndcg@2: 0.536289	valid's ndcg@3: 0.522442
Training start:                                                                  
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047858 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 47243                                               
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172
Training until validation scores don't improve for 20 rounds                     
[5]	valid's ndcg@1: 0.534857	valid's ndcg@2: 0

# milestone

In [ ]:
'lambdarank_truncation_level': 46, 'learning_rate': 0.023221403531782224, 'num_iterations': 33}

In [7]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 47,
                'learning_rate':0.023221403531782224,
                'num_iterations': 83,
                # 'lambdarank_truncation_level': 10,
                # 'learning_rate': 0.02273417953255777,
                # n_estimators は num_iterations と同義　default = 100
                # 'n_estimators': 97,
                # 'num_leaves': 42,
                'force_col_wise':True
            }



In [8]:
import warnings

warnings.filterwarnings("ignore")

ll.learn_lgb2(train,lgbm_params=lgbm_params)

[LightGBM] [Info] Total Bins 47243
[LightGBM] [Info] Number of data points in the train set: 96523, number of used features: 172


# Simulate


In [5]:
# !pwd
save_path = '/Users/Owner/Desktop/Horse/ll_obj.pickle'

def save_pickle(save_path,object_):
    with open(save_path, mode="wb") as f:
        pickle.dump(object_, f)

def load_pickle(save_path):
    with open(save_path, mode="rb") as f:
        object_ = pickle.load(f)
    return object_

# save_pickle(save_path,ll)
ll = load_pickle(save_path)
ll.learn_model_ft()


In [36]:



for odds in [1]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    fukusho_df = sl.get_result_df(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables,kaime='fukusho',odds=odds)
    tansho_df = sl.get_result_df(ll.r.data_c.loc[ll.x_test.index[0]:],return_tables,kaime='tansho',odds=odds)



odds 1


# calc results df

In [11]:
def calc_fukusho_acc(fukusho_df,odds=1.1,bet=100):
    race_num = len(fukusho_df)
    count_in3 = 0
    count_list = [0 for _ in range(3)]

    for i in range(len(fukusho_df)):
        col = fukusho_df.iloc[i]
        actual_rank_list = col['actual rank list']
        pred_list = col['pred list']
        fukusho_list = col['fukusho list']
        is_in = False
        visited = False

        for i,pred in enumerate(pred_list):
            is_in =  pred in fukusho_list
            if is_in and not visited:
                count_in3 += 1
                visited = True

            if pred_list[i] in fukusho_list:
                count_list[i] += 1
                
    # 予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合
    acc_in3 = count_in3/race_num * 100
    acc_list = [i/race_num * 100 for i in count_list]

    print("予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合",acc_in3)
    print("予測した3頭のそれぞれの複勝率",acc_list)
    

def calc_tansho_accuracy(result_df,bet=100):
    race_num = len(result_df)
    atari_num = result_df[result_df['is_atari']==True].sum()['is_atari']
    acc = (atari_num/race_num) * 100
    
    buy_num = result_df[result_df['is_buy']==True].sum()['is_buy']
    buy_df = result_df[result_df['is_buy']]
    atari_buy_num = buy_df[buy_df['is_atari']==True].sum()['is_atari']
    # 条件付き精度 (買ったもとで当たった確率)
    acc_cond = (atari_buy_num/buy_num) * 100
    total_expense = buy_num * bet
    return_price = result_df.sum()['profit']
    # 回収率
    profit_rate = (total_expense + return_price)/total_expense * 100
    atari_dist = {i:0 for i in range(1,13)}
    atari_buy_df = buy_df[buy_df['is_atari']==True]

    for race_id in atari_buy_df.index:
        race_num = int(str(race_id)[-2:])
        atari_dist[race_num] += (1/atari_buy_num) * 100
    
    print('acc ',acc)
    print('acc cond',acc_cond)
    print('profit rate',profit_rate)
    print('atari race distribution',atari_dist)



In [37]:
calc_fukusho_acc(fukusho_df)

予測した3頭の馬のうち, 1頭以上複勝圏に含まれている割合 86.20302860347728
予測した3頭のそれぞれの複勝率 [50.1402131239484, 41.61525518788559, 38.75490745933819]


In [39]:

calc_tansho_accuracy(tansho_df)


acc  22.994952327537856
acc cond 22.87323943661972
profit rate 104.82816901408452
atari race distribution {1: 10.344827586206895, 2: 8.866995073891625, 3: 8.374384236453201, 4: 9.60591133004926, 5: 8.620689655172413, 6: 8.12807881773399, 7: 10.591133004926107, 8: 7.881773399014777, 9: 7.635467980295565, 10: 6.403940886699506, 11: 5.1724137931034475, 12: 8.374384236453201}


# race_id_list

In [11]:

race_id_list = ['2022040303{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022100403{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022010203{}'.format(str(i).zfill(2)) for i in range(1,13)]



# race_id_list += ['2022050308{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022090301{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090302{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090303{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022090304{}'.format(str(i).zfill(2)) for i in range(1,13)]

# race_id_list += ['2022020103{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020104{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020105{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022020106{}'.format(str(i).zfill(2)) for i in range(1,13)]



In [48]:
race_id_list = []

for place in range(1, 11, 1):
    for kai in range(1, 6, 1):
        for day in range(1, 13, 1):
            for r in range(1, 13, 1):
                race_id = "2022" + str(place).zfill(2) + str(kai).zfill(2) +\
		str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)

In [54]:
results.index.tolist()
race_id_list = list(map(int,race_id_list))
t_ = race_id_list.copy()



[202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020906,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020907,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020908,
 202005020909,
 202005020909,
 202005020909,
 202005020909,
 202005020909,
 202005020909,
 202005020909,
 202005020909,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020910,
 202005020911,
 202005020911,
 202005020

# Predictor

In [12]:
pt = Predictor(peds,results,horse_results,race_id_list)
# ********* test_size = 0  : ずっと0にしてなかった...
pt.path_ft = ll.path_ft
pt.learn_lgb(lgbm_params,test_size=0)

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/18720 [00:00<?, ?it/s]

  0%|          | 0/18720 [00:00<?, ?it/s]

...

hr finish


  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/276 [00:00<?, ?it/s]

[LightGBM] [Info] Total Bins 49464
[LightGBM] [Info] Number of data points in the train set: 120314, number of used features: 172


# 新潟

In [ ]:
base  = '2022040303'
pt.predict(base+'12')

# 小倉

In [ ]:
base  = '2022100403'
# for i in ['07','08','09','10','11']:
pt.predict(base+'12')

# 札幌

In [ ]:
base  = '2022010203'
# for i in ['07','08','09','10','11']:
pt.predict(base+'12')

# 当日の収支計算

In [45]:
data =  ShutubaTable.scrape(race_id_list, pt.date)
st = ShutubaTable(data)
st.preprocessing()
st.merge_horse_results(pt.hr)
st.merge_peds(pt.pe.peds_vec)
st.process_categorical(pt.r.le_horse, pt.r.le_jockey, pt.r.data_pe)
pt.st = st
sl = RankSimulater(pt.model)
pred_table = sl.return_pred_table(st.data_c)
pt.sl = sl

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
sl.return_table_today(pt.race_id_list)


  0%|          | 0/36 [00:00<?, ?it/s]

return_tables　と, return_tables_todayで形式が異なるので, 形式をそろえる

# Today sim の流れ
1. return_table_today 実行して, 当日の return_table をゲット
2. その　table で回収率計算

# return_table の成形

In [95]:
"""
    col
    0 : 買目
    1 : 馬の着順
    2 : 払い戻し
    3 : 人気
"""
race_id = '202204030301'
return_table_ = return_table.loc[race_id]
rank1 = int(return_table_[return_table[0]=='単勝'][1].values[0])
real_odds = int(return_table_[return_table[0]=='単勝'][2].str.replace('円','').values[0])/100



In [69]:
sl.return_tables
race_id = '202204030301'
return_table = sl.return_tables.loc[race_id]
# return_table を成形する関数ほしい

,0,1,2,3
202204030301,単勝,6,300円,1人気
202204030301,複勝,6 11 4,110円110円140円,2人気1人気3人気
202204030301,枠連,5 8,420円,1人気
202204030301,馬連,6 11,430円,1人気
202204030301,ワイド,6 11 4 6 4 11,170円250円330円,1人気2人気3人気
202204030301,馬単,6 11,880円,1人気
202204030301,3連複,4 6 11,730円,1人気
202204030301,3連単,6 11 4,"2,350円",1人気


In [243]:

class TodaySimulater(RankSimulater):


    def __init__(self,model):
        super(TodaySimulater,self).__init__(model)
        self.is_long = False


    
    def calc_tansho(self,data_c,race_id,odds,bet,return_tables):
        race_id = int(race_id)
        pred_df = self.return_pred_table(data_c.loc[race_id],is_long=self.is_long)
        pred_df = pred_df.loc[race_id]
        pred_df = pred_df.sort_values('scores',ascending=False)
        pred_1 = pred_df['馬番'].iloc[0]
        score_1 = pred_df['scores'].iloc[0]
        score_2 = pred_df['scores'].iloc[1]
# pred_odds と actual_rank だけ, 当日用に変えればOK

        return_table = return_tables.loc[str(race_id)]
        rank1 = int(return_table[return_table[0]=='単勝'][1].values[0])
        tmp_odds = return_table[return_table[0]=='単勝'][2].str.replace(',','')
        tmp_odds = tmp_odds.str.replace('円','').values[0]
        real_odds = int(tmp_odds)/100

        not_buy_reason = ''
        profit = 0
        # try:
        #     rank = data_c[data_c['rank']==1].loc[race_id]['馬番']
        # except Exception as e:
        #     print(e)
        #     return 0,False,False,0,not_buy_reason,-1
            
        is_atari = False
        is_buy = True

        if pred_1 == rank1:
            is_atari = True

            if real_odds < odds:
                is_buy = False
                not_buy_reason = 'low odds'

#             上位２着の予測スコアが同じなら賭けない
        if score_1==score_2:
            is_buy = False
            not_buy_reason = 'same score 1 and  2'


        if is_atari:
            if is_buy:
                profit = real_odds*bet
            else:
                profit = 0
        else:
            if is_buy:
                profit -= bet
            else:
                profit = 0

        return profit, is_atari, is_buy, rank1, not_buy_reason, real_odds
    
    
    def return_race_result(self,data_c,race_id,return_tables):
        race_id = int(race_id)
        pred_df = self.return_pred_table(data_c.loc[race_id],is_long=self.is_long)
        return_table  = return_tables.loc[str(race_id)]
        pred_df = pred_df.loc[race_id]
        pred_df = pred_df.sort_values('scores',ascending=False)
        pred_list = [int(pred_df['馬番'].iloc[i]) for i in range(len(pred_df))]

        score_1 = pred_df['scores'].iloc[0]
        score_2 = pred_df['scores'].iloc[1]
        is_same_score = False

        
        tansho_row = return_table[return_table[0]=='単勝']
        fukusho_row = return_table[return_table[0]=='複勝']
        umaren_row =  return_table[return_table[0]=='馬連']
        umatan_row =  return_table[return_table[0]=='馬単']
        wide_row =  return_table[return_table[0]=='ワイド']
        sanrentan_row =  return_table[return_table[0]=='3連単']
        sanrenpuku_row =  return_table[return_table[0]=='3連複']
        
        actual_rank_list = list(map(int,sanrentan_row[1].str.split(' ').values[0]))
        
        odds_tmp =umaren_row[2].str.replace(',','')
        umaren_odds = int(odds_tmp.str.replace('円','').values[0])/100
        
        odds_tmp = tansho_row[2].str.replace(',','')
        tansho_odds = int(odds_tmp.str.replace('円','').values[0])/100
        
        odds_tmp = umatan_row[2].str.replace(',','')
        umatan_odds = int(odds_tmp.str.replace('円','').values[0])/100
        
        odds_tmp = sanrenpuku_row[2].str.replace(',','')
        sanrenpuku_odds = int(odds_tmp.str.replace('円','').values[0])/100
        
        odds_tmp = sanrentan_row[2].str.replace(',','')
        sanrentan_odds = int(odds_tmp.str.replace('円','').values[0])/100
        
        fukusho_odds = list(map(lambda x: int(x.replace(',',''))/100 , fukusho_row[2].str.split('円').values[0][0:3]))
        wide_odds = list(map(lambda x: int(x.replace(',',''))/100 , wide_row[2].str.split('円').values[0][0:3]))
        
        tmp_list = list(map(int,wide_row[1].str.split(' ').tolist()[0]))
        wide_comb = []
        for i in range(0,len(tmp_list),2):
            wide_comb.append(tmp_list[i:i+2])
        
        if score_1 == score_2:
            is_same_score =True
        
        
        return  pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb

    def get_result_df(self, data_c, return_tables, kaime='tansho', odds=2, bet=100):
        race_dict = {}

        if kaime=='tansho':
            for race_id in race_id_list:
                profit,is_atari,is_buy,actual_rank,not_buy_reason,pred_odds = self.calc_tansho(data_c,race_id,odds,bet,return_tables)
                row_list = [profit,is_atari,is_buy,actual_rank,not_buy_reason,pred_odds]
                race_dict[int(race_id)] = row_list
            tansho_result = pd.DataFrame(race_dict).T
            tansho_result.rename(columns={0:'profit',1:'is_atari',2:'is_buy',3:'actual_rank',4:'not_buy_reason',5:'real_odds'},inplace=True)
            return tansho_result
        
        if kaime=='all':
            for race_id in race_id_list:
                pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb = self.return_race_result(data_c,race_id,odds,bet,return_tables)
                row_list = [pred_list,actual_rank_list,tansho_odds,fukusho_odds,umaren_odds,wide_odds,umatan_odds,sanrenpuku_odds,sanrentan_odds,wide_comb]
                race_dict[int(race_id)] = row_list
            all_result = pd.DataFrame(race_dict).T
            all_result.rename(columns={
                0:'pred_list',
                1:'actual_rank_list',
                2:'tansho_odds',
                3:'fukusho_odds',
                4:'umaren_odds',
                5:'wide_odds',
                6:'umatan_odds',
                7:'sanrenpuku_odds',
                8:'sanrentan_odds',
                9:'wide_comb'
                },inplace=True)
            return all_result

In [158]:
tsl.return_table_today(pt.race_id_list)
tsl.return_tables

  0%|          | 0/36 [00:00<?, ?it/s]

,0,1,2,3
202204030301,単勝,6,300円,1人気
202204030301,複勝,6 11 4,110円110円140円,2人気1人気3人気
202204030301,枠連,5 8,420円,1人気
202204030301,馬連,6 11,430円,1人気
202204030301,ワイド,6 11 4 6 4 11,170円250円330円,1人気2人気3人気
...,...,...,...,...
202201020312,馬連,1 3,580円,2人気
202201020312,ワイド,1 3 1 2 2 3,240円700円920円,3人気11人気13人気
202201020312,馬単,1 3,"1,080円",3人気
202201020312,3連複,1 2 3,"2,130円",8人気


In [ ]:
# data =  ShutubaTable.scrape(race_id_list, pt.date)
# st = ShutubaTable(data)
# st.preprocessing()
# st.merge_horse_results(pt.hr)
# st.merge_peds(pt.pe.peds_vec)
# st.process_categorical(pt.r.le_horse, pt.r.le_jockey, pt.r.data_pe)


tsl = TodaySimulater(pt.model)
pred_table = tsl.return_pred_table(st.data_c)

In [156]:
tsl = TodaySimulater(pt.model)
tansho_result = tsl.get_result_df( data_c, tsl.return_tables,odds=5)


In [173]:
tansho_result

,profit,is_atari,is_buy,actual_rank,not_buy_reason,real_odds
202204030301,-100,False,True,6,,3.0
202204030302,-100,False,True,10,,1.8
202204030303,0,False,False,11,same score 1 and 2,2.0
202204030304,-100,False,True,3,,13.0
202204030305,0,False,False,15,same score 1 and 2,1.9
202204030306,-100,False,True,7,,4.6
202204030307,-100,False,True,17,,3.5
202204030308,-100,False,True,10,,3.5
202204030309,-100,False,True,15,,3.3
202204030310,-100,False,True,7,,3.3


In [167]:
return_table_ =  tsl.return_tables.loc['202204030301']
return_table_

,0,1,2,3
202204030301,単勝,6,300円,1人気
202204030301,複勝,6 11 4,110円110円140円,2人気1人気3人気
202204030301,枠連,5 8,420円,1人気
202204030301,馬連,6 11,430円,1人気
202204030301,ワイド,6 11 4 6 4 11,170円250円330円,1人気2人気3人気
202204030301,馬単,6 11,880円,1人気
202204030301,3連複,4 6 11,730円,1人気
202204030301,3連単,6 11 4,"2,350円",1人気


In [138]:
# race_id_list = list(set(data_c.index))
odds = 2.0
bet = 100
race_dict = {}
tsl = TodaySimulater(pt.model)
for race_id in race_id_list:
    profit,is_atari,is_buy,actual_rank,not_buy_reason,pred_odds = tsl.calc_tansho(data_c,race_id,odds,bet,sl.return_tables)
    row_list = [profit,is_atari,is_buy,actual_rank,not_buy_reason,pred_odds]
    race_dict[int(race_id)] = row_list
tansho_result = pd.DataFrame(race_dict).T
tansho_result.rename(columns={0:'profit',1:'is_atari',2:'is_buy',3:'actual_rank',4:'not_buy_reason',5:'real_odds'},inplace=True)

In [146]:
pred_df = tsl.return_pred_table(data_c.loc[202204030312])
pred_df

,馬番,scores
202204030312,4,0.121115
202204030312,6,0.073574
202204030312,5,-0.032901
202204030312,2,-0.066071
202204030312,10,-0.098544
202204030312,9,-0.225280
202204030312,1,-0.295827
202204030312,13,-0.355674
202204030312,15,-0.700816
202204030312,14,-0.706144


In [140]:
tansho_result

,profit,is_atari,is_buy,actual_rank,not_buy_reason,real_odds
202204030301,-100,False,True,6,,3.0
202204030302,-100,False,True,10,,1.8
202204030303,0,False,False,11,same score 1 and 2,2.0
202204030304,-100,False,True,3,,13.0
202204030305,0,False,False,15,same score 1 and 2,1.9
202204030306,-100,False,True,7,,4.6
202204030307,-100,False,True,17,,3.5
202204030308,-100,False,True,10,,3.5
202204030309,-100,False,True,15,,3.3
202204030310,-100,False,True,7,,3.3


In [56]:
,

sl.return_tables.loc['202204030301']

,0,1,2,3
202204030301,単勝,6,300円,1人気
202204030301,複勝,6 11 4,110円110円140円,2人気1人気3人気
202204030301,枠連,5 8,420円,1人気
202204030301,馬連,6 11,430円,1人気
202204030301,ワイド,6 11 4 6 4 11,170円250円330円,1人気2人気3人気
202204030301,馬単,6 11,880円,1人気
202204030301,3連複,4 6 11,730円,1人気
202204030301,3連単,6 11 4,"2,350円",1人気


# 日付に注意

In [ ]:
date = '2022/12/31'

# Results scraping

In [65]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

  0%|          | 0/5568 [00:00<?, ?it/s]

# Horse_results scraping

In [66]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)

# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

  0%|          | 0/4712 [00:00<?, ?it/s]

# Peds scraping

In [67]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

  0%|          | 0/4712 [00:00<?, ?it/s]

  0%|          | 0/4712 [00:00<?, ?it/s]

# Return scraping

In [68]:
returns_tmp = Return.scrape(race_id_list)


  0%|          | 0/5568 [00:00<?, ?it/s]

# update

In [69]:
new_results = update_data(load_csv(path_win+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_win+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_win+'peds.csv'),peds_tmp)
returns = load_csv(path_win+'return.csv')
returns.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [70]:
new_results.to_csv(path_win+'results.csv')
new_horse_results.to_csv(path_win+'horse_results.csv')
new_peds.to_csv(path_win+'peds.csv')
new_return.to_csv(path_win+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

In [ ]:
lis_ = [1,2,3,4,True,4]
lis2_ = [2,3,4,5,False,5]
dic_ = {
}
dic_['a'] = lis_
dic_['b'] = lis2_

df_ = pd.DataFrame(dic_).T

In [ ]:
df_

In [ ]:
df_.rename(columns={0:'profit',1:'is_atari',2:'is_buy',3:'actual_rank',4:'not_buy_reason',5:'is_error'})